In [4]:
import xlwings as xw
import pandas as pd


In [112]:
wb = xw.Book(r'C:\Users\SH\Desktop\ttttt\master.xlsx')
sh = wb.sheets(1)

df = sh.range('A1').options(pd.DataFrame, index=False, expand='table', dates=pd.to_datetime).value

for col in df.select_dtypes(include='float64').columns:
    df[col] = df[col].astype(int)

df['강의월'] = df['강의일자'].astype('str').str[4:6]
df.dtypes

### 공통 필터링
# df = df[(df['강의상태']=='진행') & (df['강의시간']>1)]
df

,현업교육코드,현업교육명,강의일자,강의시간,대표강사,대표강사사번,강의상태,강의월
0,1,A,20240701,2,추준성,11111111,진행,07
1,2,B,20240701,2,추준성,11111111,진행,07
2,2,B,20240703,1,이산하,22222222,진행,07
3,4,D,20240705,1,이산하,22222222,진행,07
4,2,B,20240601,2,추준성,11111111,진행,06
5,2,B,20240603,2,추준성,11111111,진행,06
6,2,B,20240501,2,추준성,11111111,진행,05
7,2,B,20240401,2,추준성,11111111,진행,04
8,4,D,20240601,1,이산하,11111111,진행,06
9,4,D,20240501,1,이산하,11111111,진행,05


In [60]:
### 공통 필터링 (강의상태, 대표강사 여부)
case1_df = df[df['강의월']=='07']
case1_df

,현업교육코드,현업교육명,강의일자,강의시간,대표강사,대표강사사번,강의월
0,1,A,20240701,2,추준성,11111111,07
1,2,B,20240701,2,추준성,11111111,07
2,2,B,20240703,1,이산하,22222222,07
3,4,D,20240705,1,이산하,22222222,07


In [61]:
wb = xw.Book(r'C:\Users\SH\Desktop\ttttt\pool.xlsx')
sh = wb.sheets(1)

pool_df = sh.range('A1').options(pd.DataFrame, index=False, expand='table', dates=pd.to_datetime).value

for col in pool_df.select_dtypes(include='float64').columns:
    pool_df[col] = pool_df[col].astype(int)

pool_df

,현업교육코드,현업교육명
0,1,A
1,2,B
2,3,C


In [134]:
### case 1. 지급대상으로 선정된 학습조직, 강의 월만 가져오기
case1_df = case1_df[case1_df['현업교육코드'].isin(pool_df['현업교육코드'])]
case1_df=case1_df.assign(최종대상='1.지급대상')

case1_df

,현업교육코드,현업교육명,강의일자,강의시간,대표강사,대표강사사번,강의월,최종대상
0,1,A,20240701,2,추준성,11111111,07,1.지급대상
1,2,B,20240701,2,추준성,11111111,07,1.지급대상
2,2,B,20240703,1,이산하,22222222,07,1.지급대상


In [114]:
#######################
### case 2: 이번에 새로 3개월 채운 학습조직
case2_df = df[df['강의월'].isin(['05', '06', '07'])] ### TODO 자동으로 3개월 구성

case2_df = case2_df[~ case2_df['현업교육코드'].isin(pool_df['현업교육코드'])] ### 지급대상이 아니고 
case2_df

,현업교육코드,현업교육명,강의일자,강의시간,대표강사,대표강사사번,강의상태,강의월
3,4,D,20240705,1,이산하,22222222,진행,07
8,4,D,20240601,1,이산하,11111111,진행,06
9,4,D,20240501,1,이산하,11111111,진행,05
13,5,E,20240501,1,eee,9999999,진행,05
14,5,E,20240699,1,eee,9999999,진행,06
15,6,F,20240705,1,이산하,22222222,준비,07
16,6,F,20240601,1,이산하,11111111,진행,06
17,6,F,20240501,1,이산하,11111111,진행,05


In [115]:
pivot = case2_df.pivot_table(index='현업교육코드', columns='강의월', values='현업교육명', aggfunc='count')
pivot = pivot.fillna(0).astype(int)
pivot.dtypes

강의월
05    int64
06    int64
07    int64
dtype: object

In [131]:
filtered_code = pivot[(pivot>0).all(axis=1)].index ### 3개월 연속을 만족하는 현업교육 코드 추출
print(filtered_code)

p_case2_df = case2_df[case2_df['현업교육코드'].isin(filtered_code)]
p_case2_df=p_case2_df.assign(최종대상='2.일괄_대상')

f_case2_df = case2_df[~case2_df['현업교육코드'].isin(filtered_code)]
f_case2_df=f_case2_df.assign(최종대상='2.일괄_X')

f_case2_df

Index([4, 6], dtype='int64', name='현업교육코드')


,현업교육코드,현업교육명,강의일자,강의시간,대표강사,대표강사사번,강의상태,강의월,최종대상
13,5,E,20240501,1,eee,9999999,진행,05,2.일괄_X
14,5,E,20240699,1,eee,9999999,진행,06,2.일괄_X


In [132]:
new_case2_df = pd.concat([f_case2_df,p_case2_df])

new_case2_df

,현업교육코드,현업교육명,강의일자,강의시간,대표강사,대표강사사번,강의상태,강의월,최종대상
13,5,E,20240501,1,eee,9999999,진행,05,2.일괄_X
14,5,E,20240699,1,eee,9999999,진행,06,2.일괄_X
3,4,D,20240705,1,이산하,22222222,진행,07,2.일괄_대상
8,4,D,20240601,1,이산하,11111111,진행,06,2.일괄_대상
9,4,D,20240501,1,이산하,11111111,진행,05,2.일괄_대상
15,6,F,20240705,1,이산하,22222222,준비,07,2.일괄_대상
16,6,F,20240601,1,이산하,11111111,진행,06,2.일괄_대상
17,6,F,20240501,1,이산하,11111111,진행,05,2.일괄_대상
